In [3]:
"""
Robust Stock Data Fetcher
-------------------------
Fetches historical stock data from Yahoo Finance, standardizes the format,
validates data quality, and saves to CSV with timestamp.
"""

# Standard Library Imports
from pathlib import Path
from datetime import datetime

# Third-Party Imports
import numpy as np
import pandas as pd
import yfinance as yf

# ======================
# CONFIGURATION SETTINGS
# ======================
DATA_PATH = Path("C:/Users/Aislay/bootcamp_Ziyi_Yang/homework/stage04_data-acquisition-and-ingestion/data/raw")
TICKER = "TSLA"  # Tesla stock symbol
PERIOD = "3mo"   # 3 months historical data
INTERVAL = "1d"  # Daily intervals

# =================
# CORE FUNCTIONS
# =================

def initialize_environment():
    """Ensure data directory exists"""
    DATA_PATH.mkdir(parents=True, exist_ok=True)
    return datetime.now().strftime("%Y%m%d-%H%M")

def fetch_stock_data(ticker, period, interval):
    """Download stock data from Yahoo Finance"""
    print(f"Fetching {ticker} data for {period} period...")
    df = yf.download(
        ticker, 
        period=period,
        interval=interval,
        auto_adjust=False,
        progress=False
    )
    
    if df is None or len(df) == 0:
        raise RuntimeError("Failed to download data (check network/ticker).")
    return df

def standardize_data(df):
    """Normalize dataframe structure and columns"""
    # Flatten MultiIndex columns if present
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [c[0] for c in df.columns]
    
    # Reset index and clean column names
    df = df.reset_index()
    df.columns = [str(c).strip().lower() for c in df.columns]
    
    # Standardize date column name
    date_col = next(
        (c for c in ["date", "datetime", "index"] if c in df.columns),
        df.columns[0]
    )
    df = df.rename(columns={date_col: "date"})
    
    # Standardize adjusted close column names
    rename_rules = {
        "adj close": "adjusted_close",
        "adj_close": "adjusted_close",
        "adjusted close": "adjusted_close"
    }
    df = df.rename(columns={
        k: v for k, v in rename_rules.items() 
        if k in df.columns
    })
    
    return df

def validate_data(df):
    """Ensure required columns exist with proper data types"""
    required_cols = [
        "date", "open", "high", "low", 
        "close", "adjusted_close", "volume"
    ]
    
    # Add missing columns with NaN values
    for col in required_cols:
        if col not in df.columns:
            df[col] = np.nan
    
    # Convert data types
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    numeric_cols = ["open", "high", "low", "close", "adjusted_close", "volume"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    return df[required_cols].copy()

def save_dataset(df, ticker, timestamp):
    """Save processed data to CSV"""
    filename = f"api_yfinance_{ticker}_{timestamp}.csv"
    outpath = DATA_PATH / filename
    df.to_csv(outpath, index=False)
    print(f"Data successfully saved to:\n{outpath}")
    return outpath

# =================
# MAIN EXECUTION
# =================

if __name__ == "__main__":
    try:
        # Initialize
        timestamp = initialize_environment()
        
        # Extract
        raw_data = fetch_stock_data(TICKER, PERIOD, INTERVAL)
        
        # Transform
        standardized_data = standardize_data(raw_data)
        validated_data = validate_data(standardized_data)
        
        # Load
        output_path = save_dataset(validated_data, TICKER, timestamp)
        
        # Preview
        print("\nData Preview:")
        display(validated_data.head())
        
    except Exception as e:
        print(f"\nError occurred: {str(e)}")
        raise

Fetching TSLA data for 3mo period...
Data successfully saved to:
C:\Users\Aislay\bootcamp_Ziyi_Yang\homework\stage04_data-acquisition-and-ingestion\data\raw\api_yfinance_TSLA_20250822-2102.csv

Data Preview:


,date,open,high,low,close,adjusted_close,volume
0,2025-05-23,337.920013,343.179993,333.209991,339.339996,339.339996,84654800
1,2025-05-27,347.350006,363.790009,347.320007,362.890015,362.890015,120146400
2,2025-05-28,364.839996,365.000000,355.910004,356.899994,356.899994,91404300
3,2025-05-29,365.290009,367.709991,356.000000,358.429993,358.429993,88087800
4,2025-05-30,355.519989,363.679993,345.290009,346.459991,346.459991,123474900


In [4]:
# Complete Web Scraping Code for Financial Companies Data
from pathlib import Path
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from IPython.display import display

# 1. Setup Paths and Parameters
# --------------------------------------------------
# Specify custom save path (modify according to your actual path)
CUSTOM_PATH = Path("C:/Users/Aislay/bootcamp_Ziyi_Yang/homework/homework4/data/raw")
CUSTOM_PATH.mkdir(parents=True, exist_ok=True)

# Generate timestamp automatically
STAMP = datetime.now().strftime("%Y%m%d-%H%M")

# 2. Select Target Webpage (Financial Companies List)
# --------------------------------------------------
URL = "https://en.wikipedia.org/wiki/List_of_largest_financial_services_companies_by_revenue"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 3. Web Scraping and Table Parsing
# --------------------------------------------------
try:
    print(f"Scraping financial companies webpage: {URL}")
    
    # Send HTTP request
    response = requests.get(URL, headers=HEADERS, timeout=15)
    response.raise_for_status()  # Check for HTTP errors
    
    # Parse HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find target table (select table containing financial companies data)
    # This page has multiple tables, we select the first complete data table
    target_tables = soup.find_all('table', {'class': 'wikitable'})
    if not target_tables:
        raise ValueError("No qualified table found on the page")
    
    # Typically the first table is the main data table
    table = target_tables[0]
    print("Successfully found financial companies table, parsing...")
    
    # Extract table headers
    headers = []
    for th in table.find('tr').find_all('th'):  # First row contains headers
        header_text = th.text.strip()
        # Clean special characters and line breaks
        header_text = re.sub(r'[\n\r]+', ' ', header_text)
        headers.append(header_text)
    
    # Extract table data
    table_data = []
    for row in table.find_all('tr')[1:]:  # Skip header row
        row_data = []
        for cell in row.find_all(['th', 'td']):
            cell_text = cell.text.strip()
            # Clean special characters and line breaks
            cell_text = re.sub(r'[\n\r]+', ' ', cell_text)
            row_data.append(cell_text)
        if row_data:  # Ignore empty rows
            table_data.append(row_data)
    
    # 4. Create DataFrame
    # --------------------------------------------------
    df = pd.DataFrame(table_data, columns=headers)
    
    # 5. Data Cleaning and Validation (Optimized for Financial Data)
    # --------------------------------------------------
    print("\n=== Data Validation ===")
    
    # Clean column names
    df.columns = [col.lower().replace(' ', '_').replace('.', '') for col in df.columns]
    print(f"\nCleaned column names: {df.columns.tolist()}")
    
    # Financial-specific cleaning
    # Ensure company name column exists
    if 'company' not in df.columns and 'name' not in df.columns:
        # Try to identify company name column
        company_col = next((col for col in df.columns if 'company' in col or 'name' in col), df.columns[0])
        df = df.rename(columns={company_col: 'company'})
    
    # Identify and convert numeric columns (financial-specific fields)
    numeric_cols = []
    financial_keywords = ['revenue', 'assets', 'profit', 'market_cap', 'capitalization']
    
    for col in df.columns:
        # Financial numeric fields processing
        if any(keyword in col for keyword in financial_keywords):
            try:
                # Remove currency symbols, units (B,M), thousand separators etc.
                temp_series = df[col].str.replace(r'[^\d.-]', '', regex=True)
                temp_series = pd.to_numeric(temp_series, errors='coerce')
                if not temp_series.isna().all():  # Only keep if conversion is successful
                    df[col] = temp_series
                    numeric_cols.append(col)
                    # Add unit conversion note (e.g., B for billion)
                    if 'b' in df[col].iloc[0].lower() if pd.notna(df[col].iloc[0]) else False:
                        df[col] = df[col] * 1e9  # Assuming B stands for billion
            except Exception as e:
                print(f"Error converting column {col}: {str(e)}")
                pass
    
    print(f"\nConverted financial numeric columns: {numeric_cols}")
    
    # Financial sector classification
    if 'industry' not in df.columns and 'sector' not in df.columns:
        # Try to identify financial subsectors from company names or descriptions
        df['financial_sector'] = 'Other Financial'
        bank_keywords = ['bank', 'banc', 'credit']
        insurance_keywords = ['insurance', 'assurance', 'reinsurance']
        investment_keywords = ['investment', 'asset management', 'capital', 'brokerage']
        
        def classify_sector(name):
            name = str(name).lower()
            if any(keyword in name for keyword in bank_keywords):
                return 'Banking'
            elif any(keyword in name for keyword in insurance_keywords):
                return 'Insurance'
            elif any(keyword in name for keyword in investment_keywords):
                return 'Investment Services'
            return 'Other Financial'
        
        df['financial_sector'] = df['company'].apply(classify_sector)
    
    # NA value statistics
    print("\nNA values per column:")
    print(df.isna().sum())
    
    # Data shape
    print(f"\nFinal data shape: {df.shape}")
    
    # 6. Save Raw Data
    # --------------------------------------------------
    output_filename = f"scrape_wikipedia_financial_{STAMP}.csv"
    outpath = CUSTOM_PATH / output_filename
    df.to_csv(str(outpath), index=False)
    
    print("\n=== Scraping Results ===")
    print(f"Successfully saved to: {outpath}")
    print(f"Contains {len(df)} financial companies records")
    print(f"Financial sector distribution:")
    print(df['financial_sector'].value_counts() if 'financial_sector' in df.columns else "No sector classification generated")
    print("\nData preview:")
    display(df.head())

except requests.exceptions.RequestException as e:
    print(f"\nNetwork request error: {str(e)}")
    raise
except Exception as e:
    print(f"\nProcessing error occurred: {str(e)}")
    raise

Scraping financial companies webpage: https://en.wikipedia.org/wiki/List_of_largest_financial_services_companies_by_revenue
Successfully found financial companies table, parsing...

=== Data Validation ===

Cleaned column names: ['rank', 'company', 'industry', 'revenue_(usd_millions)', 'net_income_(usd_millions)', 'total_assets_(usd_billions)', 'headquarters']
Error converting column revenue_(usd_millions): 'numpy.int64' object has no attribute 'lower'
Error converting column total_assets_(usd_billions): 'numpy.int64' object has no attribute 'lower'

Converted financial numeric columns: ['revenue_(usd_millions)', 'total_assets_(usd_billions)']

NA values per column:
rank                           0
company                        0
industry                       0
revenue_(usd_millions)         0
net_income_(usd_millions)      0
total_assets_(usd_billions)    0
headquarters                   0
dtype: int64

Final data shape: (52, 7)

=== Scraping Results ===
Successfully saved to: C:\Us

,rank,company,industry,revenue_(usd_millions),net_income_(usd_millions),total_assets_(usd_billions),headquarters
0,1,Transamerica Corporation,Insurance,245510,"42,521",873,United States
1,2,Ping An Insurance Group,Insurance,191509,"20,738",1460,China
2,3,ICBC,Banking,182794,"45,783",5110,China
3,4,China Construction Bank,Banking,172000,"39,282",4311,China
4,5,Agricultural Bank of China,Banking,153884,"31,293",4169,China
